#Prediksi Harga Daging Sapi

#Business Understanding

Proyek ini bertujuan untuk menyediakan prediksi harga daging sapi di Provinsi Jawa Timur untuk periode tertentu (misalnya, hari berikutnya) secara akurat. Informasi ini diharapkan memberikan manfaat signifikan bagi konsumen, peternak, pedagang, dan pemerintah daerah sebagai berikut:

Konsumen dapat menggunakan prediksi harga untuk merencanakan belanja dengan lebih efektif, khususnya bagi rumah tangga yang ingin mengatur anggaran belanja daging sapi, terutama saat mendekati hari besar. Mereka juga bisa membeli daging sebelum harga diprediksi naik, sehingga menghemat pengeluaran.

Peternak dapat memanfaatkan prediksi untuk menentukan waktu yang tepat dalam memotong sapi dan mendistribusikan daging ke pasar, memastikan mereka mendapatkan harga yang optimal. Dengan prediksi ini, risiko kerugian akibat penurunan harga atau stok berlebih dapat diminimalkan, sehingga meningkatkan efisiensi produksi dan distribusi.

Pedagang akan terbantu dalam mengelola stok dan menetapkan harga jual yang kompetitif berdasarkan prediksi harga. Informasi ini memungkinkan pedagang untuk mengatur persediaan secara optimal dan merencanakan strategi promosi atau diskon pada waktu yang tepat, sehingga mereka tetap dapat memenuhi kebutuhan pasar tanpa mengurangi margin keuntungan.


#Data Understanding

data yang saya ambil merupakan data time series univariates yaitu data harga Daging Sapi perhari yang diambil dari tanggal 10 oktober 2022 sampai 29 November 2024 pada provinsi jawa timur dengan jumlah data 548 dan memiliki 2 fitur:tanggal,Daging Sapi tipe data tanggal adalah time temporal dan Daging Sapi adalah numeric

In [ ]:
import pandas as pd

# Membaca file Excel
file_path = '/content/sample_data/Tabel Harga Berdasarkan Daerah (3).xlsx'  # Ganti dengan path file Excel Anda
df = pd.read_excel(file_path)

df.head()

,Date,Daging Sapi
0,10/ 10/ 2022,116650
1,11/ 10/ 2022,116900
2,12/ 10/ 2022,116900
3,13/ 10/ 2022,116800
4,14/ 10/ 2022,116900


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548 entries, 0 to 547
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Date         548 non-null    object
 1   Daging Sapi  548 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 8.7+ KB


**Deskripsi Data:**

Date = merupakan tanggal pada harga Daging sapi

Daging sapi = merupakan nilai harga dari Daging sapi

In [ ]:
df.shape

(548, 2)

Data terdiri dari 548 baris dan 2 kolom

In [ ]:
df.describe()

,Daging Sapi
count,548.000000
mean,118654.562044
std,1710.954380
min,116350.000000
25%,117437.500000
50%,118300.000000
75%,120200.000000
max,123750.000000


#Data Preprocessing

##Normalisasi Data

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import joblib

# Membaca file Excel
file_path = '/content/sample_data/Tabel Harga Berdasarkan Daerah (3).xlsx'  # Ganti dengan path file Excel Anda
df = pd.read_excel(file_path)

# Inisialisasi MinMaxScaler
scaler = MinMaxScaler()

# Menampilkan nilai minimum dan maksimum sebelum normalisasi
min_values_before = df.min()
max_values_before = df.max()

print("Nilai minimum sebelum normalisasi:")
print(min_values_before)

print("\nNilai maksimum sebelum normalisasi:")
print(max_values_before)
# Melakukan normalisasi pada kolom numerik
kolom_numerik = df.select_dtypes(include=['float64', 'int64']).columns
df[kolom_numerik] = scaler.fit_transform(df[kolom_numerik])

scaler_file_path = '/content/sample_data/scalerp1.pkl'
joblib.dump(scaler, scaler_file_path)
# Menyimpan data yang sudah dinormalisasi ke file Excel baru
combined_file_path = '/content/sample_data/normalisasi.xlsx'
df.to_excel(combined_file_path, index=False)

Nilai minimum sebelum normalisasi:
Date           01/ 01/ 2024
Daging Sapi          116350
dtype: object

Nilai maksimum sebelum normalisasi:
Date           31/ 10/ 2024
Daging Sapi          123750
dtype: object


Disini kami menormalisasi data dulu agar nanti presentase errornya kecil dan kami menggunakan metode minmaxscaler untuk normalisasinya dimana data ini memiliki nilai minimum adalah  116350 dan nilai maksimumnya adalah 123750

##Mencari Outlier

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

# Langkah 1: Baca data dari Excel atau dataset
file_path = '/content/sample_data/normalisasi.xlsx'  # Ganti dengan path file Excel Anda
df = pd.read_excel(file_path)
# Langkah 2: Pilih kolom numerik yang ingin diuji outlier-nya
numerical_columns = ['Daging Sapi']  # Ganti dengan kolom yang relevan

# Langkah 3: Hitung Z-Score untuk setiap fitur numerik
z_scores = np.abs(stats.zscore(df[numerical_columns]))

# Menentukan threshold, misalnya 3
threshold = 3

# Identifikasi outlier
outliers_zscore = np.where(z_scores > threshold)

# Menampilkan indeks dan kolom outlier
print("Indeks dan kolom yang mengandung outlier (Z-Score):")
print(outliers_zscore)

# Menampilkan data outlier
print("Data outlier berdasarkan Z-Score:")
print(df.iloc[outliers_zscore[0]])


Indeks dan kolom yang mengandung outlier (Z-Score):
(array([], dtype=int64), array([], dtype=int64))
Data outlier berdasarkan Z-Score:
Empty DataFrame
Columns: [Date, Daging Sapi]
Index: []


jadi output diatas memberitahukan bahwa data ini tidak memiliki outlier

##Sliding window

In [ ]:
# Creating lag features for multivariate time series data (PM10 and PM2.5)
# Adding lag of 1 and lag of 2 for both PM10 and PM2.5
file_path = '/content/sample_data/normalisasi.xlsx'  # Ganti dengan path file Excel Anda
data = pd.read_excel(file_path)


data['Daging Sapi_1'] = data['Daging Sapi'].shift(1)
data['Daging Sapi_2'] = data['Daging Sapi'].shift(2)
data['Daging Sapi_3'] = data['Daging Sapi'].shift(3)
data['Daging Sapi_4'] = data['Daging Sapi'].shift(4)

data_with_lag = data.dropna()

output_file_path = '/content/sample_data/hasil_sliding_window1.xlsx'  # Ganti dengan path output file Excel
data_with_lag.to_excel(output_file_path, index=False)


karena ini adalah data time series univariate maka kita akan melakukan sliding windows dulu, nah disini kami membagi fiturnya menjadi 4 sehingga data ini memiliki fitur 4 dan targetnya 1

##Pembagian Data Test dan Data Train

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the uploaded file
file_path = '/content/sample_data/hasil_sliding_window1.xlsx'
data = pd.read_excel(file_path)

# Menyiapkan data (X, y)
X = data.drop(columns=['Daging Sapi', 'Date'])
y = data['Daging Sapi']

# Menyimpan kolom tanggal untuk perbandingan nanti
dates = data['Date']

# Split data into training and testing sets
X_train, X_test, y_train, y_test, train_dates, test_dates = train_test_split(
    X, y, dates, test_size=0.2, random_state=42)

print(f"Training set size: {X_train.shape[0]}")
print(f"Testing set size: {X_test.shape[0]}")
# Simpan data yang dipisah ke dalam satu file Excel dengan sheet berbeda
output_file = '/content/sample_data/bagi.xlsx'
with pd.ExcelWriter(output_file) as writer:
    train_dates,X_train.to_excel(writer, sheet_name='X_train', index=False)
    test_dates,X_test.to_excel(writer, sheet_name='X_test', index=False)
    train_dates,y_train.to_excel(writer, sheet_name='y_train', index=False)
    test_dates,y_test.to_excel(writer, sheet_name='y_test', index=False)


output_file


Training set size: 435
Testing set size: 109


'/content/sample_data/bagi.xlsx'

sebelum melakukan modelling disini kami membagi datanya menjadi data train dan data test yaitu X_train X_test dan y_train y_test, pada data train berjumlah 435 dan data test 109

#Data Modelling

##SVR

In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

scaler_file_path = '/content/sample_data/scalerp1.pkl'
scaler = joblib.load(scaler_file_path)
# Path file (ganti dengan path file Anda sendiri)
file_path = '/content/sample_data/bagi.xlsx'

# Memuat data
data = pd.read_excel(file_path)

# Mendefinisikan dan melatih model SVR
svr_model = SVR(kernel='rbf', C=1.0, epsilon=0.1)  # Parameter dapat disesuaikan sesuai kebutuhan
svr_model.fit(X_train, y_train)

# Memprediksi pada set pengujian
y_pred = svr_model.predict(X_test)

# Menghitung dan mencetak metrik evaluasi
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f"MAE: {mae}, RMSE: {rmse}, MAPE: {mape}")

# --- Prediksi untuk hari berikutnya (11-10-2024) menggunakan 30 data terakhir ---
# Mengambil 30 data terakhir untuk prediksi
user_input = {
    "Daging Sapi_1": 18500,  # Ganti dengan nilai aktual fitur Anda
    "Daging Sapi_2": 27111,
    "Daging Sapi_3": 34000,
    "Daging Sapi_4": 23000  # Sesuaikan jumlah fitur dengan dataset Anda
}

# Konversi input user ke DataFrame
user_input_df = pd.DataFrame([user_input])

user_input_normalized = pd.DataFrame(
    scaler.transform(user_input_df),  # Menggunakan scaler yang dimuat
    columns=user_input_df.columns
)
# Lakukan prediksi
predicted_price = svr_model.predict(user_input_normalized)
print(f"\nPrediksi harga berdasarkan input user: {predicted_price[0]}")

min_harga = 116350 # Nilai minimum harga dalam dataset asli
max_harga = 123750  # Nilai maksimum harga dalam dataset asli

# Denormalisasi hasil prediksi harga untuk semua data dalam array
predicted_next_day_prices_original = predicted_price * (max_harga - min_harga) + min_harga


# Menghitung rata-rata harga prediksi (denormalisasi)
predicted_next_day_price_average = np.mean(predicted_next_day_prices_original)
print(f"\nPrediksi rata-rata harga (denormalisasi) untuk 11-10-2024: Rp {predicted_next_day_price_average:,.2f}")
# --- Membandingkan prediksi dengan data aktual ---
# Membuat DataFrame untuk nilai aktual vs prediksi
comparison_df = pd.DataFrame({
    "tanggal": X_test.index,
    "Actual": y_test,
    "Predicted": y_pred
})

# Mengurutkan DataFrame berdasarkan tanggal
comparison_df = comparison_df.sort_values(by="tanggal").reset_index(drop=True)

print("\nHarga Aktual vs Prediksi:")
print(comparison_df)


MAE: 0.05824881376062393, RMSE: 0.07642164938499454, MAPE: 0.9032953748793611

Prediksi harga berdasarkan input user: 0.5768509324088328

Prediksi rata-rata harga (denormalisasi) untuk 11-10-2024: Rp 120,618.70

Harga Aktual vs Prediksi:
     tanggal    Actual  Predicted
0          0  0.074324   0.119287
1          2  0.040541   0.103919
2          6  0.040541   0.103752
3          9  0.040541   0.103752
4         10  0.040541   0.103752
..       ...       ...        ...
104      528  0.743243   0.705076
105      530  0.621622   0.752988
106      532  0.743243   0.614173
107      539  0.743243   0.748763
108      542  0.743243   0.731312

[109 rows x 3 columns]


##Decision Tree Regressor

In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

# Path file (ganti dengan path file Anda sendiri)
file_path = '/content/sample_data/bagi.xlsx'

# Memuat data
data = pd.read_excel(file_path)

# Mendefinisikan dan melatih model Decision Tree
dt_model = DecisionTreeRegressor(random_state=42, max_depth=5)  # Parameter dapat disesuaikan
dt_model.fit(X_train, y_train)

# Memprediksi pada set pengujian
y_pred = dt_model.predict(X_test)

# Menghitung dan mencetak metrik evaluasi
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f"MAE: {mae}, RMSE: {rmse}, MAPE: {mape}")

# --- Prediksi untuk hari berikutnya (11-10-2024) menggunakan 30 data terakhir ---
# Mengambil 30 data terakhir untuk prediksi
last_30_data = X.iloc[-30:]

# Memprediksi harga hari berikutnya dengan merata-rata prediksi dari 30 data terakhir
predicted_next_day_prices = dt_model.predict(last_30_data)
predicted_next_day_price = np.mean(predicted_next_day_prices)
print(f"\nPrediksi harga untuk 11-10-2024: {predicted_next_day_price}")

min_harga = 116350 # Nilai minimum harga dalam dataset asli
max_harga = 123750  # Nilai maksimum harga dalam dataset asli

# Denormalisasi hasil prediksi harga untuk semua data dalam array
predicted_next_day_prices_original = predicted_next_day_prices * (max_harga - min_harga) + min_harga


# Menghitung rata-rata harga prediksi (denormalisasi)
predicted_next_day_price_average = np.mean(predicted_next_day_prices_original)
print(f"\nPrediksi rata-rata harga (denormalisasi) untuk 11-10-2024: Rp {predicted_next_day_price_average:,.2f}")
# --- Membandingkan prediksi dengan data aktual ---
# Membuat DataFrame untuk nilai aktual vs prediksi
comparison_df = pd.DataFrame({
    "tanggal": X_test.index,
    "Actual": y_test,
    "Predicted": y_pred
})

# Mengurutkan DataFrame berdasarkan tanggal
comparison_df = comparison_df.sort_values(by="tanggal").reset_index(drop=True)

print("\nHarga Aktual vs Prediksi:")
print(comparison_df)


MAE: 0.0279241223183897, RMSE: 0.06140912530660946, MAPE: 0.16234274377770241

Prediksi harga untuk 11-10-2024: 0.7175454866631341

Prediksi rata-rata harga (denormalisasi) untuk 11-10-2024: Rp 121,659.84

Harga Aktual vs Prediksi:
     tanggal    Actual  Predicted
0          0  0.074324   0.042885
1          2  0.040541   0.042885
2          6  0.040541   0.042885
3          9  0.040541   0.042885
4         10  0.040541   0.042885
..       ...       ...        ...
104      528  0.743243   0.666534
105      530  0.621622   0.738739
106      532  0.743243   0.666534
107      539  0.743243   0.738739
108      542  0.743243   0.738739

[109 rows x 3 columns]


##Random Forest Regressor

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

# Path file (ganti dengan path file Anda sendiri)
file_path = '/content/sample_data/bagi.xlsx'

# Memuat data
data = pd.read_excel(file_path)

# Mendefinisikan dan melatih model Decision Tree
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Menghitung dan mencetak metrik evaluasi
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f"MAE: {mae}, RMSE: {rmse}, MAPE: {mape}")

# --- Prediksi untuk hari berikutnya (11-10-2024) menggunakan 30 data terakhir ---
# Mengambil 30 data terakhir untuk prediksi
last_30_data = X.iloc[-30:]

# Memprediksi harga hari berikutnya dengan merata-rata prediksi dari 30 data terakhir
predicted_next_day_prices = dt_model.predict(last_30_data)
predicted_next_day_price = np.mean(predicted_next_day_prices)
print(f"\nPrediksi harga untuk 11-10-2024: {predicted_next_day_price}")

min_harga = 116350 # Nilai minimum harga dalam dataset asli
max_harga = 123750  # Nilai maksimum harga dalam dataset asli

# Denormalisasi hasil prediksi harga untuk semua data dalam array
predicted_next_day_prices_original = predicted_next_day_prices * (max_harga - min_harga) + min_harga


# Menghitung rata-rata harga prediksi (denormalisasi)
predicted_next_day_price_average = np.mean(predicted_next_day_prices_original)
print(f"\nPrediksi rata-rata harga (denormalisasi) untuk 11-10-2024: Rp {predicted_next_day_price_average:,.2f}")
# --- Membandingkan prediksi dengan data aktual ---
# Membuat DataFrame untuk nilai aktual vs prediksi
comparison_df = pd.DataFrame({
    "tanggal": X_test.index,
    "Actual": y_test,
    "Predicted": y_pred
})

# Mengurutkan DataFrame berdasarkan tanggal
comparison_df = comparison_df.sort_values(by="tanggal").reset_index(drop=True)

print("\nHarga Aktual vs Prediksi:")
print(comparison_df)


MAE: 0.02601543665074973, RMSE: 0.06192344767713792, MAPE: 0.14423941669821153

Prediksi harga untuk 11-10-2024: 0.7175454866631341

Prediksi rata-rata harga (denormalisasi) untuk 11-10-2024: Rp 121,659.84

Harga Aktual vs Prediksi:
     tanggal    Actual  Predicted
0          0  0.074324   0.041757
1          2  0.040541   0.038851
2          6  0.040541   0.040203
3          9  0.040541   0.040203
4         10  0.040541   0.040203
..       ...       ...        ...
104      528  0.743243   0.672896
105      530  0.621622   0.733216
106      532  0.743243   0.651327
107      539  0.743243   0.732743
108      542  0.743243   0.743243

[109 rows x 3 columns]


##XGBRegressor

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

scaler_file_path = '/content/sample_data/scalerp1.pkl'
scaler = joblib.load(scaler_file_path)
# Path file (ganti dengan path file Anda sendiri)
file_path = '/content/sample_data/bagi.xlsx'

# Memuat data
data = pd.read_excel(file_path)

# Mendefinisikan dan melatih model Decision Tree
model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Menghitung dan mencetak metrik evaluasi
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f"MAE: {mae}, RMSE: {rmse}, MAPE: {mape}")

user_input = {
    "Daging Sapi_1": 122200,  # Ganti dengan nilai aktual fitur Anda
    "Daging Sapi_2": 121111,
    "Daging Sapi_3": 124000,
    "Daging Sapi_4": 120000  # Sesuaikan jumlah fitur dengan dataset Anda
}

# Konversi input user ke DataFrame
user_input_df = pd.DataFrame([user_input])

user_input_normalized = pd.DataFrame(
    scaler.transform(user_input_df),  # Menggunakan scaler yang dimuat
    columns=user_input_df.columns
)
# Lakukan prediksi
predicted_price = model.predict(user_input_normalized)
print(f"\nPrediksi harga berdasarkan input user: {predicted_price[0]}")

# Contoh: Asumsikan normalisasi menggunakan Min-Max Scaling
min_harga = 116350 # Nilai minimum harga dalam dataset asli
max_harga = 123750  # Nilai maksimum harga dalam dataset asli

# Denormalisasi hasil prediksi harga untuk semua data dalam array
predicted_next_day_prices_original = predicted_price * (max_harga - min_harga) + min_harga


# Menghitung rata-rata harga prediksi (denormalisasi)
predicted_next_day_price_average = np.mean(predicted_next_day_prices_original)
print(f"\nPrediksi rata-rata harga (denormalisasi) untuk 11-10-2024: Rp {predicted_next_day_price_average:,.2f}")
# --- Membandingkan prediksi dengan data aktual ---
# Membuat DataFrame untuk nilai aktual vs prediksi
comparison_df = pd.DataFrame({
    "tanggal": X_test.index,
    "Actual": y_test,
    "Predicted": y_pred
})

# Mengurutkan DataFrame berdasarkan tanggal
comparison_df = comparison_df.sort_values(by="tanggal").reset_index(drop=True)

print("\nHarga Aktual vs Prediksi:")
print(comparison_df)


MAE: 0.025995623318262554, RMSE: 0.05880313698421077, MAPE: 0.14195117560899198

Prediksi harga berdasarkan input user: 0.6760862469673157

Prediksi rata-rata harga (denormalisasi) untuk 11-10-2024: Rp 121,353.04

Harga Aktual vs Prediksi:
     tanggal    Actual  Predicted
0          0  0.074324   0.044226
1          2  0.040541   0.039052
2          6  0.040541   0.040894
3          9  0.040541   0.040894
4         10  0.040541   0.040894
..       ...       ...        ...
104      528  0.743243   0.668162
105      530  0.621622   0.740362
106      532  0.743243   0.611872
107      539  0.743243   0.705294
108      542  0.743243   0.741926

[109 rows x 3 columns]


pada model XGBRegressor ini memperlihatkan bahwa model ini memiliki nilai error yang kecil dibandingkan beberapa model yang kami coba yaitu memiliki MAE:0,0259 RMSE:0,0588 MAPE:0,1419